In [79]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath('RS/thinker/thinker')
if module_path not in sys.path:
    sys.path.append(module_path)

from collections import namedtuple
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
from collections import deque
import time
import numpy as np
import argparse
import torch
import torch.nn.functional as F
from thinker.env import Environment
from thinker.net import ActorNet, ModelNet
from thinker.buffer import ModelBuffer
import thinker.util as util

flags = util.parse([])
flags.model_batch_size = 2
flags.model_buffer_n = 1000
flags.model_warm_up_n = 500
flags.model_batch_mode = False
flags.model_rnn = True
flags.model_unroll_length = 8
t = flags.model_unroll_length   
flags.model_k_step_return = 5
k = flags.model_k_step_return
flags.actor_parallel_n = 1
n = flags.actor_parallel_n  

flags.model_batch_mode = True

P = namedtuple("P", ["x","y"])
model_buffer = ModelBuffer(flags)

c = 0
for c in range(100):
    data = P(torch.full((t+k, n, 1),c), torch.full((t+k, n, 1),c+0.1))    
    state = (torch.full((n, 3),c+0.2), torch.full((n, 3),c+0.3))
    model_buffer.write(data, state, np.random.randint(10))

print(model_buffer.next_inds)
data, data_state, weights, abs_flat_inds, inds = model_buffer.read(1)
print(data.x[:, :, -1], data_state)

state = tuple(torch.zeros_like(x) for x in data_state)
priorities = torch.zeros(len(abs_flat_inds))

model_buffer.update_priority(abs_flat_inds, priorities, state)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[ 1. 26.  4.  7. 13. 32. 21.  8. 15. 14. 11. 16. 20. 23. 18. 17. 28. 47.
 22. 37. 25. 36. 24. 27. 30. 42. 29. 61. 46. 34. 33. 39. 48. 35. 40. 63.
 38. 55. 43. 50. 41. 52. 53. 44. 45. 49. 58. 51. 54. 56. 88. 57. 59. 67.
 62. 71. 76. 64. 69. 60. 65. 78. 75. 66. 70. 74. 68. 81. 83. 79. 80. 72.
 73. 77. 82. nan 90. 85. 93. 89. 87. 86. 95. 84. nan 94. 91. 98. nan 97.
 96. 92. nan nan nan nan nan nan 99. nan]
tensor([[23, 27],
        [23, 27],
        [23, 27],
        [23, 27],
        [23, 27],
        [23, 27],
        [23, 27],
        [23, 27]]) (tensor([[23.2000, 23.2000, 23.2000],
        [27.2000, 27.2000, 27.2000]]), tensor([[23.3000, 23.3000, 23.3000],
        [27.3000, 27.3000, 27.3000]]))


In [82]:
model_buffer.state[int(62-model_buffer.base_ind/n)]

(tensor([[62.2000, 62.2000, 62.2000]]), tensor([[62.3000, 62.3000, 62.3000]]))

In [106]:
from thinker.env import Environment
from thinker.net import ModelNet

flags = util.parse([])
flags.model_rnn = True

model_net = ModelNet((3,80,80), 5, flags)
print(model_net.rnn)

env = Environment(flags)
env_out, model_state = env.initial(model_net)
print("1", model_state[0][0,0,0,0])

a = torch.tensor([1,1,1]).long().unsqueeze(0).unsqueeze(0)
for n in range(1, 40 * 150):
    env_out, model_state = env.step(a, model_net, model_state)
    print(n, model_state[0][0,0,0,0])

True
1 tensor([ 0.0038,  0.0181,  0.0199,  0.0221,  0.0209,  0.0206,  0.0236, -0.0042])
1 tensor([ 0.0038,  0.0181,  0.0199,  0.0221,  0.0209,  0.0206,  0.0236, -0.0042])
2 tensor([ 0.0038,  0.0181,  0.0199,  0.0221,  0.0209,  0.0206,  0.0236, -0.0042])
3 tensor([ 0.0038,  0.0181,  0.0199,  0.0221,  0.0209,  0.0206,  0.0236, -0.0042])
4 tensor([ 0.0038,  0.0181,  0.0199,  0.0221,  0.0209,  0.0206,  0.0236, -0.0042])
5 tensor([ 0.0038,  0.0181,  0.0199,  0.0221,  0.0209,  0.0206,  0.0236, -0.0042])
6 tensor([ 0.0038,  0.0181,  0.0199,  0.0221,  0.0209,  0.0206,  0.0236, -0.0042])
7 tensor([ 0.0038,  0.0181,  0.0199,  0.0221,  0.0209,  0.0206,  0.0236, -0.0042])
8 tensor([ 0.0038,  0.0181,  0.0199,  0.0221,  0.0209,  0.0206,  0.0236, -0.0042])
9 tensor([ 0.0038,  0.0181,  0.0199,  0.0221,  0.0209,  0.0206,  0.0236, -0.0042])
10 tensor([ 0.0038,  0.0181,  0.0199,  0.0221,  0.0209,  0.0206,  0.0236, -0.0042])
11 tensor([ 0.0038,  0.0181,  0.0199,  0.0221,  0.0209,  0.0206,  0.0236, -0.0042